In [6]:
from tvpblp import TwoStepGMM_BLP
TwoStepGMM_BLP()

TypeError: TwoStepGMM_BLP.__init__() missing 1 required positional argument: 'data'

In [1]:
%load_ext autoreload
%autoreload 2

import mypkg.tvblp as tvblp   # module import (not "from … import …")

import pandas as pd
import numpy as np
from numpy.linalg import inv

In [2]:
df = pd.read_csv('data_yoghurt.csv')


# Normalize attributes by weight
df['calories'] = df['calories'] / df['weight']
df['sugar'] = df['sugar'] / df['weight']
df['protein'] = df['protein'] / df['weight']

# Compute outside good share
df = df.join(pd.DataFrame(1 - df.groupby(by=['city', 'month']).share.sum()).rename(columns={'share': 'og_share'}), on=['city', 'month'])

#Add intercept
df['const'] = 1

# Prepare y and X
y = np.log(df.share).values - np.log(df.og_share).values
X = df[['const', 'price', 'weight', 'calories', 'sugar', 'protein']].values



df['sugar_dev'] = df.sugar - df[(df.month==1)&(df.city==1)].sugar.mean()
df['market'] = 'C'+(df.city).astype(str)+'M'+(df.month).astype(str)
df = df.join(df.groupby(by = 'market').sugar.sum().rename('instrument_2'), on = 'market')
df['instrument_1'] = df['diesel']*df['distance']

prod = df['product'].copy()  # stash original
df = pd.get_dummies(
    df,
    columns=['city', 'month', 'product'],
    prefix=['city', 'month', 'product'],
    drop_first=True
)
df.insert(0, 'product', prod)  # re-add original (position 0 optional)

lin_columns = df.columns.drop(['share', 'weight', 'calories', 'sugar', 'protein', 'og_share', 'instrument_1', 'distance', 'diesel', 'sugar_dev', 'market', 'instrument_2', 'product'])
random_columns = ['sugar_dev']

In [3]:
model = tvblp.TwoStepGMM_BLP(df, share_col='share', price_col='price',
lin = lin_columns, random = random_columns,
instruments = ['instrument_1', 'instrument_2'], market_id='market',
product_id='product', integration_points=50, integration_int = [0.1, 0.9], og_share_col='og_share')

Initialized BLP with 27584 observations across 7192 markets.
Linear characteristics: ['price', 'const', 'city_2', 'city_3', 'city_4', 'city_5', 'city_6', 'city_7', 'city_8', 'city_9', 'city_10', 'city_11', 'city_12', 'city_13', 'city_14', 'city_15', 'city_16', 'city_17', 'city_18', 'city_19', 'city_20', 'city_21', 'city_22', 'city_23', 'city_24', 'city_25', 'city_26', 'city_27', 'city_28', 'city_29', 'city_30', 'city_31', 'city_32', 'city_33', 'city_34', 'city_35', 'city_36', 'city_37', 'city_38', 'city_39', 'city_40', 'city_41', 'city_42', 'city_43', 'city_44', 'city_45', 'city_46', 'city_47', 'city_48', 'city_49', 'city_50', 'city_51', 'city_52', 'city_53', 'city_54', 'city_55', 'city_56', 'city_57', 'city_58', 'city_59', 'city_60', 'city_61', 'city_62', 'city_63', 'city_64', 'city_65', 'city_66', 'city_67', 'city_68', 'city_69', 'city_70', 'city_71', 'city_72', 'city_73', 'city_74', 'city_75', 'city_76', 'city_77', 'city_78', 'city_79', 'city_80', 'city_81', 'city_82', 'city_83', 'c

In [4]:
W = inv(model.Z.T@model.Z)
model.fit(W, L_init = [[700]])


Starting optimization with L_init diagonal: [700.]
Stopping tolerance: max|ΔL| < 1e-12
  Iter   5: obj = 1.350198e-03, max|ΔL| = 4.731e+02
  Iter  10: obj = 2.036775e-19, max|ΔL| = 3.116e-04
  Converged! max|ΔL| = 3.695e-13 < 1e-12

Optimization finished after 13 iterations
Convergence by L change: True
Success: True
Final objective: 3.061705e-23
Optimal L:
[[152.8598705]]


{'L_opt': array([[152.8598705]]),
 'obj_val': 3.0617051503614194e-23,
 'result':   message: CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL
   success: True
    status: 0
       fun: 3.061705150361419e-43
         x: [ 1.529e+02]
       nit: 9
       jac: [ 0.000e+00]
      nfev: 13
      njev: 13
  hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>,
 'n_iter': 13}

In [5]:
list([1,2,3])

[1, 2, 3]